In [21]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 

#만들어놓은 그리드월드 환경을 불러온다.
from lib.envs.gridworld import GridworldEnv

In [24]:
env=GridworldEnv()


np.argmax관련 링크입니다. 


https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.argmax.html

# Solution

In [25]:

def value_iteration(env, theta=0.0001, discount_factor=1.0):
    
    
    
    
    def one_step_lookahead(state, V):#한 스테이트 뒤를 예측하는 함수 생성
        
         
        """션
        Helper function to calculate the value for all action in a given state.
        
        Args:
        
            state: 고려하는 스테이트 The state to consider (int)
            V: 예측 값, 벡터의 길이는 env.nS이다. 
            
            The value to use as an estimator, Vector of length env.nS
        
        Returns:
       각 엑션의 예측값을 포함한 env.nA 길이의 벡터를 리턴
        
            A vector of length env.nA containing the expected value of each action.
        """
        
        
        A = np.zeros(env.nA)
        
        #모든 엑션
        for a in range(env.nA):
            #어떠한 스테이트에서 엑션의 확률, 다음스테이트, 리워드, 끝을  for으로 돌린다
            for prob, next_state, reward, done in env.P[state][a]:
                
                A[a] += prob * (reward + discount_factor * V[next_state])
                
        return A
    #모든 스테이트의 벨류는 0부터 시작
    V = np.zeros(env.nS)
    while True:
        # Stopping condition
        
        delta = 0
        #각 스테이트를 업데이트 한다
        
        # Update each state...
        for s in range(env.nS):
            
            # Do a one-step lookahead to find the best action
            #가장 좋은 엑션을 찾아서 어사인
            A = one_step_lookahead(s, V)
            
            best_action_value = np.max(A)
            
            # Calculate delta across all states seen so far
            
            delta = max(delta, np.abs(best_action_value - V[s]))
            
            # Update the value function
            
            V[s] = best_action_value        
        # Check if we can stop 
        
        if delta < theta:
            break
    
    # Create a deterministic policy using the optimal value function
    policy = np.zeros([env.nS, env.nA])
    for s in range(env.nS):
        # One step lookahead to find the best action for this state
        A = one_step_lookahead(s, V)
        best_action = np.argmax(A)
        # Always take the best action
        policy[s, best_action] = 1.0
    
    return policy, V

In [27]:
%%timeit
value_iteration(env)

610 µs ± 16.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [16]:
policy, v = value_iteration(env)

print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Policy Probability Distribution:
[[ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]

